Ideas 1:
1. first we need to convert our sparse matrix to dense matrix where each row is an example by keeping only the diagonals.
2. Change in Shapes:
   1. for 1D models: 
      1. the shape will change from (nxn) to (nx3) since each cell has 3 terms (i.e. 2 neighbors and the cell itself). 
      2. Formulation:
         1.  b, c1, c2 x x1 > b1
         2. c1, c2, c3 x x2 > b2
         3. c2, c3, c4 x x3 > b3
         4. c3, c4, b  x x4 > b4
   2. for 2D models: 
      1. the shape will change from (nxn) to (nx5) since each cell has 5 terms (i.e. 4 neighbors and the cell itself).  
   3. for 3D models:
      1. the shape will change from (nxn) to (nx7) since each cell has 7 terms (i.e. 6 neighbors and the cell itself).

Idea 2: Train neural network to calc A^-1 then use that to get x: A^-1.b = x

In [1]:
import openresim as ors
import numpy as np
import pandas as pd
import tensorflow as tf
import keras

In [18]:
def create_model():
    grid = ors.grids.Cartesian(
        nx=4, ny=1, nz=1, dx=300, dy=350, dz=40, phi=0.27, kx=270, dtype="double"
    )
    fluid = ors.fluids.SinglePhase(mu=0.5, B=1, dtype="double")
    model = ors.models.Model(grid, fluid, dtype="double", verbose=False)
    model.set_well(id=4, q=-600, s=1.5, r=3.5)
    model.set_boundaries({0: ("pressure", 4000), 5: ("rate", 0)})
    return model

model = create_model()

In [19]:
[print(f"cell {k}: {dict(v[0])}") for k, v in model.get_cells_eq().items()]

cell 1: {p1: 85.2012000000000, p2: -28.4004000000000}
cell 2: {p1: 28.4004000000000, p3: 28.4004000000000, p2: -56.8008000000000}
cell 3: {p2: 28.4004000000000, p4: 28.4004000000000, p3: -56.8008000000000}
cell 4: {p3: 28.4004000000000, p4: -28.4004000000000}


[None, None, None, None]

In [20]:
A, d = model.get_matrices_vectorized(sparse=False)

In [21]:
list(model.pressures[0])

[4000.0, nan, nan, nan, nan, nan]

In [22]:
model.run(
        nsteps=3,
        sparse=True,
        threading=True,
        vectorize=True,
        check_MB=True,
        print_arrays=True,
        isolver="cgs")

[info] Simulation run started: 3 timesteps.


 33%|███▎      | 1/3 [00:00<00:00,  3.47steps/s]

step: 0
[[ 85.2012 -28.4004   0.       0.    ]
 [ 28.4004 -56.8008  28.4004   0.    ]
 [  0.      28.4004 -56.8008  28.4004]
 [  0.       0.      28.4004 -28.4004]
 [-85.2012  28.4004   0.       0.    ]
 [ 28.4004 -56.8008  28.4004   0.    ]
 [  0.      28.4004 -56.8008  28.4004]
 [  0.       0.      28.4004 -28.4004]
 [170.4024 -56.8008   0.       0.    ]
 [  0.       0.       0.       0.    ]
 [  0.       0.       0.       0.    ]
 [  0.       0.       0.       0.    ]]
[[ 227203.2 -227203.2  454406.4]
 [      0.        0.        0. ]
 [      0.        0.        0. ]
 [    600.      600.        0. ]]

step: 1
[[ 85.2012 -28.4004   0.       0.    ]
 [ 28.4004 -56.8008  28.4004   0.    ]
 [  0.      28.4004 -56.8008  28.4004]
 [  0.       0.      28.4004 -28.4004]
 [-85.2012  28.4004   0.       0.    ]
 [ 28.4004 -56.8008  28.4004   0.    ]
 [  0.      28.4004 -56.8008  28.4004]
 [  0.       0.      28.4004 -28.4004]
 [170.4024 -56.8008   0.       0.    ]
 [  0.       0.       0.      

100%|██████████| 3/3 [00:00<00:00,  3.87steps/s]

step: 2
[[ 85.2012 -28.4004   0.       0.    ]
 [ 28.4004 -56.8008  28.4004   0.    ]
 [  0.      28.4004 -56.8008  28.4004]
 [  0.       0.      28.4004 -28.4004]
 [-85.2012  28.4004   0.       0.    ]
 [ 28.4004 -56.8008  28.4004   0.    ]
 [  0.      28.4004 -56.8008  28.4004]
 [  0.       0.      28.4004 -28.4004]
 [170.4024 -56.8008   0.       0.    ]
 [  0.       0.       0.       0.    ]
 [  0.       0.       0.       0.    ]
 [  0.       0.       0.       0.    ]]
[[ 227203.2 -227203.2  454406.4]
 [      0.        0.        0. ]
 [      0.        0.        0. ]
 [    600.      600.        0. ]]

[info] Simulation run of 3 steps finished in 0.78 seconds.

[info] Material Balance Error: -1.4909971923771081e-07.


In [23]:
list(model.pressures)

[array([4000.,   nan,   nan,   nan,   nan,   nan]),
 array([4000.        , 3989.4367685 , 3968.31030549, 3947.18384248,
        3926.05737947,           nan]),
 array([4000.        , 3989.4367685 , 3968.31030549, 3947.18384248,
        3926.05737947,           nan]),
 array([4000.        , 3989.4367685 , 3968.31030549, 3947.18384248,
        3926.05737947,           nan])]

In [24]:
A

array([[-85.2012,  28.4004,   0.    ,   0.    ],
       [ 28.4004, -56.8008,  28.4004,   0.    ],
       [  0.    ,  28.4004, -56.8008,  28.4004],
       [  0.    ,   0.    ,  28.4004, -28.4004]])

In [25]:
d

array([[-227203.2],
       [      0. ],
       [      0. ],
       [    600. ]])

In [123]:
np.concatenate([
    np.concatenate([[0], np.diag(A, -1)]).reshape(-1,1),
    np.diag(A, 0).reshape(-1,1),
    np.concatenate([np.diag(A, 1), [0]]).reshape(-1,1)],
    axis=1
)
df = pd.DataFrame()
df['L'] = np.concatenate([[0], np.diag(A, -1)])
df['C'] = np.diag(A, 0)
df['R'] = np.concatenate([np.diag(A, 1), [0]])
df['B'] = -d
# df['P_t0'] = model.pressures[0, 1:-1]
# df['P_t1'] = model.pressures[1, 1:-1]
df['P_L'] = model.pressures[1, 0:-2]
df['P_C'] = model.pressures[1, 1:-1]
df['P_R'] = model.pressures[1, 2:]
df.replace(np.nan, 0, inplace=True)
df


,L,C,R,B,P_L,P_C,P_R
0,0.0000,-85.2012,28.4004,227203.2,4000.000000,3989.436768,3968.310305
1,28.4004,-56.8008,28.4004,-0.0,3989.436768,3968.310305,3947.183842
2,28.4004,-56.8008,28.4004,-0.0,3968.310305,3947.183842,3926.057379
3,28.4004,-28.4004,0.0000,-600.0,3947.183842,3926.057379,0.000000


In [124]:
df.iloc[:, 0:3].values[np.newaxis]

array([[[  0.    , -85.2012,  28.4004],
        [ 28.4004, -56.8008,  28.4004],
        [ 28.4004, -56.8008,  28.4004],
        [ 28.4004, -28.4004,   0.    ]]])

In [128]:
r = 0
x0 = keras.Input(shape=(3,), name='input')
x1 = keras.layers.Dense(1, name='layer_1')(x0)
nn_model = keras.Model(inputs=x0, outputs=x1)
nn_model.weights

[<tf.Variable 'layer_1/kernel:0' shape=(3, 1) dtype=float32, numpy=
 array([[ 0.9246787 ],
        [-1.0246596 ],
        [-0.95987284]], dtype=float32)>,
 <tf.Variable 'layer_1/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

In [129]:
nn_model.get_layer('layer_1').set_weights([
    df.iloc[:2, :3].values.reshape(3,-1),
    np.array(df.loc[:2, 'B'])[np.newaxis],
])
nn_model.weights

ValueError: Layer layer_1 weight shape (3, 1) is not compatible with provided weight shape (3, 2).

In [126]:
nn_model.predict(df.loc[r, ['P_L','P_C','P_R']].values[np.newaxis])

1/1 [==============================] - 0s 110ms/step


array([[-0.015625]], dtype=float32)

In [77]:
x0 = keras.Input(shape=(3,), name='input')
x1 = keras.layers.Dense(4, name='layer_1')(x0)
x2 = keras.layers.Dense(1, name='layer_2')(x1)
nn_model = keras.Model(inputs=x0, outputs=x2)
# nn_model.predict(df.iloc[:, :4].values)
# nn_model.predict(df[['P_L','P_C','P_R']])

In [82]:
nn_model.weights

[<tf.Variable 'layer_1/kernel:0' shape=(3, 4) dtype=float32, numpy=
 array([[  0.    ,  28.4004,  28.4004,  28.4004],
        [-85.2012, -56.8008, -56.8008, -28.4004],
        [ 28.4004,  28.4004,  28.4004,   0.    ]], dtype=float32)>,
 <tf.Variable 'layer_1/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'layer_2/kernel:0' shape=(4, 1) dtype=float32, numpy=
 array([[-1.0167873 ],
        [ 0.7982335 ],
        [-0.26296145],
        [ 0.09138703]], dtype=float32)>,
 <tf.Variable 'layer_2/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

In [84]:
nn_model.get_layer('layer_1').set_weights([
    df.iloc[:, :3].values.transpose(),
    df['B'].values,
]
    )#set_weights()

In [86]:
nn_model.predict(df[['P_L','P_C','P_R']])

1/1 [==============================] - 0s 72ms/step


array([[ -187.98238],
       [-1220.1295 ],
       [-2440.2744 ],
       [49740.383  ]], dtype=float32)

In [ ]:
x0 = keras.Input(shape=(4,3), name='input')
x1 = keras.layers.Dense(2, name='layer_1')(x0)
x2 = keras.layers.Dense(1, name='layer_2')(x1)
model = keras.Model(inputs=x0, outputs=x2)
model.predict(df.iloc[:, 0:3].values[np.newaxis])

In [ ]:
model.weights